In [1]:
# Widgets
import bqviz as bqv
from ipywidgets import *
from bqwidgets import *
from bqplot import *
import multiple_select
import importlib
importlib.reload(multiple_select) 
from traitlets import link
from bqplot_extra.regression_lines import PolyFit

# first step is to import the bql module
import bql
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import warnings
warnings.filterwarnings("ignore")

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

In [2]:
def bq_res_array_to_pd(res_array):
    a = res_array[0].df().reset_index()
    r_all = pd.DataFrame({'ID':a['ID']})
    for res in res_array:
        r_n = res.df().reset_index()
        r_all = r_all.merge(r_n, on = 'ID')
    return r_all

def see_substrat (tgt = None):
    sub_strategies.options = mat_list_strat.loc[mat_list_strat['Estrategia'].isin([strategies.value]), 'Subestrategia'].tolist()
    
    #Update values in accordions
    accordion_geo_ms.update_values(data.loc[data['Estrategia'].isin([strategies.value]) & data['Subestrategia'].isin([sub_strategies.value]), 'Area Geográfica 2'].unique().tolist(), True)
    accordion_risk_ms.update_values(data.loc[data['Estrategia'].isin([strategies.value]) & data['Subestrategia'].isin([sub_strategies.value]), 'Perfil de Riesgo'].unique().tolist(), True)    
    
def Download_data (tgt = None):
    global main_df    
    
    aux_down_mat = data.loc[~data['Ticker BLG'].str.contains('Index')]
    aux_down_mat = aux_down_mat.loc[aux_down_mat['Estrategia'].isin([strategies.value]) &
                                    data['Subestrategia'].isin([sub_strategies.value]) &
                                    aux_down_mat['Perfil de Riesgo'].isin(accordion_risk_ms.read_values()) &
                                    aux_down_mat['Area Geográfica 2'].isin(accordion_geo_ms.read_values())]
    
    sec_list = aux_down_mat['Ticker BLG'].tolist()
    
    req = bql.Request(sec_list, {'Patrimonio actual': aum['value'],
                                 'WTD': wtd['value'], 'MTD': mtd['value'], 'YTD': ytd['value'],
                                 'Volatilidad 3M': vol3m['value'], 'Volatilidad 1Y': vol1y['value'], 'Downside Vol 3M': d_vol3m['value'], 'Downside Vol 1Y': d_vol1y['value'],
                                 'Drawdown': ddwan['value']})                                          
    
    res = bq.execute(req)
    
    main_df = bq_res_array_to_pd(res)   
    
def Filter_Data (tgt = None):
    global main_df
    global main_df_filter

    geo_list = accordion_geo_ms.read_values()
    risk_list = accordion_risk_ms.read_values()
    
    geo_list = accordion_geo_ms.read_values()
    risk_list = accordion_risk_ms.read_values()

    main_df_filter = pd.merge(main_df, data[['Ticker BLG', 'Area Geográfica 2', 'Perfil de Riesgo']].rename(columns = {'Ticker BLG': 'ID'}), how='left', on=['ID'])

    if geo_list == []:
        main_df_filter = main_df_filter.loc[main_df_filter['Perfil de Riesgo'].isin(risk_list)]
    elif risk_list == []:
        main_df_filter = main_df_filter.loc[main_df_filter['Area Geográfica 2'].isin(geo_list)]
    else:
        main_df_filter = main_df_filter.loc[main_df_filter['Perfil de Riesgo'].isin(risk_list) & main_df_filter['Area Geográfica 2'].isin(geo_list)]

    main_df_filter = main_df_filter.drop(['Area Geográfica 2', 'Perfil de Riesgo'], axis=1)    
    
def press_main_button (tgt = None):
    
    Download_data()
    Filter_Data()
    Generate_Output1()
    Generate_Output2()   
    tab.layout.visibility = 'visible'
    
def accordion_call_back(tgt = None):

    Filter_Data()
    Generate_Output1()
    Generate_Output2()
        
def build_accordion(name, width, margin, cb_func):
    accordion = Accordion()
    accordion.selected_index = None
    accordion.set_title(0, name)
    accordion.layout.width = '300px'
    
    ms = multiple_select.MultipleSelect(width, margin, cb_func)
    accordion.children = [ms.panel]
    return accordion, ms    

def update_output1 (tgt = None):
    
    Generate_Output1()
    
def update_output2 (tgt = None):

    Generate_Output2()

def Generate_Output1 (tgt = None):
    global main_df_filter    
    
    x = main_df_filter['ID'].tolist()
    y = main_df_filter[d_stats.value].tolist()
    
    #Plot
    bar.x = []
    bar.y = []
    bar.color = []
    
    bar.x = x
    bar.y = y
    bar.color = y  
    
    tooltip_bar.labels = ['Fondo', d_stats.value]
    
def Generate_Output2 (tgt = None):
    global main_df_filter    
    
    x = main_df_filter[x_stats.value].tolist()
    y = main_df_filter[y_stats.value].tolist()
    labels = main_df_filter['ID'].tolist()
    
    #Scatter
    scatt.x = []
    scatt.y = []
    scatt.names = []
    
    scatt.x = x
    scatt.y = y
    scatt.names = labels
    
    tooltip_scatt.labels = [x_stats.value, y_stats.value]
    
    p.x_train = x
    p.y_train = y

In [3]:
aux_h = widgets.Label(value='')
aux_h.layout.width = '15px'

aux_v = widgets.Label(value='')
aux_v.layout.width = '5px'

#Load the data
data = pd.read_excel('Soporte/securities.xlsx')

##############################################################################################

global_strat = data[['Ticker BLG', 'Estrategia']]

aum = f.znav(d.FUND_TOTAL_ASSETS(fill = 'prev', currency = 'EUR'))
agg_aum = f.sum(f.group(aum))

date_today = datetime.today()
last_year = datetime(date_today.year - 1, 12, 31)
last_month = datetime(date_today.year, date_today.month, 1) - relativedelta(days = 1)
past_5d = date_today - relativedelta(days = 5)

ytd = f.dropna(d.TOTAL_RETURN(calc_interval = f.range(last_year, date_today), currency = 'EUR'))
mtd = f.dropna(d.TOTAL_RETURN(calc_interval = f.range(last_month, date_today), currency = 'EUR'))
wtd = f.dropna(d.TOTAL_RETURN(calc_interval = f.range(past_5d, date_today), currency = 'EUR'))

agg_ytd = f.avg(f.group(ytd))
agg_mtd = f.avg(f.group(mtd))
agg_wtd = f.avg(f.group(wtd))

past_3m = datetime(date_today.year, date_today.month - 2, 1) - relativedelta(days = 1)
past_1y = date_today - relativedelta(years = 1)

vol3m = f.dropna(d.VOLATILITY(calc_interval = f.range(past_3m, date_today), currency = 'EUR'))
vol1y = f.dropna(d.VOLATILITY(calc_interval = f.range(past_1y, date_today), currency = 'EUR'))

agg_vol3m = f.avg(f.group(vol3m))
agg_vol1y = f.avg(f.group(vol1y))

d_vol3m = f.dropna(d.DOWNSIDE_VOLATILITY(calc_interval = f.range(past_3m, date_today), currency = 'EUR'))
d_vol1y = f.dropna(d.DOWNSIDE_VOLATILITY(calc_interval = f.range(past_1y, date_today), currency = 'EUR'))

agg_d_vol3m = f.avg(f.group(d_vol3m))
agg_d_vol1y = f.avg(f.group(d_vol1y))

ddwan = f.dropna(d.Max_Drawdown(calc_interval = f.range(past_1y, date_today), currency = 'EUR'))

agg_ddwan = f.avg(f.group(ddwan))

agg_global_mat = pd.DataFrame(data={})
for i_strat in global_strat['Estrategia'].unique():
    
    fil_strat = global_strat.loc[global_strat['Estrategia'].isin([i_strat]), 'Ticker BLG'].to_frame()
    fil_strat = fil_strat.loc[~fil_strat['Ticker BLG'].str.contains('Index')]
    
    req = bql.Request(fil_strat['Ticker BLG'].tolist(), {'AUM': agg_aum['value'],
                                                         'WTD': agg_wtd['value'], 'MTD': agg_mtd['value'], 'YTD': agg_ytd['value'],
                                                         'VOL3M': agg_vol3m['value'], 'VOL1Y': agg_vol1y['value'], 'DOWN_VOL3M': agg_d_vol3m['value'], 'DOWN_VOL1Y': agg_d_vol1y['value'],
                                                         'DRAWDOWN': agg_ddwan['value']})
    res = bq.execute(req)
    
    i_aux_mat = bq_res_array_to_pd(res)
    i_aux_mat['ID'] = i_aux_mat['ID'].replace('IdentityGroup', i_strat)
    
    agg_global_mat = agg_global_mat.append(i_aux_mat, ignore_index=True)
    
agg_global_mat['AUM_rel'] = agg_global_mat['AUM'] / sum(agg_global_mat['AUM'])

##############################################################################################

# Path
path = widgets.Text(value=os.getcwd() + '\Soporte', disabled=True)
path.layout.width = '650px'

# Label Overview
label_overview = widgets.HTML('<font size=5><b> RESUMEN </b></font>')

###############  Plot Overview

#Pie chart 1
pie_data1 = np.array(agg_global_mat['AUM_rel'].tolist())

pie1_sc = ColorScale()
tooltip_widget1 = Tooltip(fields=['size'], formats=['0.3%'], labels = ['AUM (%)'])
pie1 = Pie(sizes=pie_data1, scales={'color': pie1_sc}, color=pie_data1,
          tooltip=tooltip_widget1, interactions = {'hover': 'tooltip'})

pie1.label_color = 'white'
pie1.font_size = '11px'
pie1.display_labels = 'outside'
pie1.radius = 170
pie1.inner_radius = 100
pie1.start_angle = -90
pie1.end_angle = 90

pie1.labels = [n +'  '+'{:0.1%}'.format(d) for n,d in zip(agg_global_mat['ID'].tolist(), pie_data1)]
fig_pie1 = Figure(marks=[pie1], title = 'Patrimonio (%)', title_style = {'font-size': '25px', 'font-weight': 'bold', 'text-decoration': 'underline', 'fill': 'white'})
fig_pie1.layout.width = '685px'
fig_pie1.layout.height = '260px'
fig_pie1.fig_margin = {'top': 0, 'bottom': -200, 'left': 0, 'right': 0}

#Pie chart 2
pie_data2 = np.array(agg_global_mat['AUM'].tolist())

pie2_sc = ColorScale()
tooltip_widget2 = Tooltip(fields=['size'], formats=['0€'], labels = ['AUM (€)'])
pie2 = Pie(sizes=pie_data2, scales={'color': pie2_sc}, color=pie_data2,
          tooltip=tooltip_widget2, interactions = {'hover': 'tooltip'})

pie2.label_color = 'white'
pie2.font_size = '11px'
pie2.display_labels = 'outside'
pie2.radius = 170
pie2.inner_radius = 100
pie2.start_angle = -90
pie2.end_angle = 90

pie2.labels = [n +'  '+'{:.1f}'.format(d/1000000000) + ' B€' for n,d in zip(agg_global_mat['ID'].tolist(), pie_data2)]
fig_pie2 = Figure(marks=[pie2], title = 'Patrimonio (Billones €)', title_style = {'font-size': '25px', 'font-weight': 'bold', 'text-decoration': 'underline', 'fill': 'white'})
fig_pie2.layout.width = '685px'
fig_pie2.layout.height = '260px'
fig_pie2.fig_margin = {'top': 0, 'bottom': -200, 'left': 0, 'right': 0}

#Returns Bar Chart
over_returns = bqv.BarPlot(agg_global_mat[['ID', 'WTD', 'MTD', 'YTD']].set_index(['ID']), padding=.7)                  
 
over_returns.figure.title = 'Retornos'
over_returns.figure.title_style = {'font-size': '25px', 'font-weight': 'bold', 'text-decoration': 'underline', 'fill': 'white'}
over_returns.figure.layout.width = '850px'
over_returns.figure.layout.height = '260px'
over_returns.yaxis.tick_format = '0.2%'
over_returns.figure.legend_location = 'top-left'
over_returns.mark.tooltip = Tooltip(fields=['x', 'y'], formats=['','0.3%'], labels=['Categoría', 'Retorno'])

#Volatility Bar Chart
over_vol = bqv.BarPlot(agg_global_mat[['ID', 'VOL3M', 'VOL1Y', 'DOWN_VOL3M', 'DOWN_VOL1Y']].set_index(['ID']), padding=.7)                  

over_vol.figure.title = 'Volatilidades'
over_vol.figure.title_style = {'font-size': '25px', 'font-weight': 'bold', 'text-decoration': 'underline', 'fill': 'white'}
over_vol.figure.layout.width = '850px'
over_vol.figure.layout.height = '260px'
over_vol.yaxis.tick_format = '0.2%'
over_vol.figure.legend_location = 'top-left'
over_vol.mark.tooltip = Tooltip(fields=['x', 'y'], formats=['','0.3%'], labels=['Categoría', 'Volatilidad'])

#Drawdown Scatter Chart
dd_sc_x = LinearScale()
dd_sc_y = LogScale()
dd_sc_e = LinearScale()
dd_sc_col = ColorScale()

tooltip_widget3 = Tooltip(fields=['y'], formats=['0.3%'], labels = ['Drawdown'])
scatt = Scatter(x=[1]*(len(agg_global_mat)), y=agg_global_mat['DRAWDOWN'].tolist(), names=[n +'  '+'{:.3f}'.format(d) + ' %' for n,d in zip(agg_global_mat['ID'].tolist(), agg_global_mat['DRAWDOWN'].tolist())],
                scales={'x': dd_sc_x, 'y': dd_sc_y, 'color': dd_sc_col, 'rotation': dd_sc_e}, tooltip=tooltip_widget3, marker= 'circle', color = agg_global_mat['DRAWDOWN'].tolist())

over_dd = Figure(marks=[scatt], title='Drawdown', title_style = {'font-size': '25px', 'font-weight': 'bold', 'text-decoration': 'underline', 'fill': 'white'}, layout = Layout(width = '180px', height = '520px'))
over_dd.fig_margin = {'top': 70, 'bottom': 30, 'left': 10, 'right': 0}

row0 = widgets.HBox([widgets.VBox([fig_pie1, fig_pie2]), widgets.VBox([over_returns.show(), over_vol.show()]), over_dd])

##################

# Label Analysis
label_analysis = widgets.HTML('<font size=5><b> ANÁLISIS </b></font>')

# Label Strategy
label_strategies = widgets.Label(value="Tipo de estrategia")
label_strategies.layout.width = '300px'

# DropDown 
strategies = widgets.Dropdown(options=['Valor Relativo',
                                       'Selección de Títulos ',
                                       'Global Macro',
                                       'Multiestrategia',
                                       'Materias Primas',
                                       'Eventos Corporativos',
                                       'CTA'], value = 'Valor Relativo')
strategies.layout.width = '300px'
strategies.observe(see_substrat, names = 'value')

mat_list_strat = data[['Estrategia', 'Subestrategia']].drop_duplicates()

# Label Sub-strategy
label_Sub_strategies = widgets.Label(value="Tipo de Subestrategia")
label_Sub_strategies.layout.width = '315px'

# DropDown 
sub_strategies = widgets.Dropdown(options = mat_list_strat.loc[mat_list_strat['Estrategia'].isin(['Valor Relativo']), 'Subestrategia'].tolist())
sub_strategies.layout.width = '315px'

# Button
button1 = widgets.Button(description='CARGAR',button_style='warning')
# button1.layout.height = '124px'
button1.on_click(press_main_button)

# Row1
row1 = widgets.VBox([widgets.HBox([label_strategies, aux_h, label_Sub_strategies]), widgets.HBox([strategies, aux_h, sub_strategies, button1])])

# Label Filters
label_filters = widgets.HTML('<font size=3><i> Filtros </i></font>')

# Geography accordion
accordion_geo, accordion_geo_ms = build_accordion('Geografía', '100%', '1px', accordion_call_back)

# Risk accordion
accordion_risk, accordion_risk_ms = build_accordion('Perfil de Riesgo', '100%', '1px', accordion_call_back)

#Update values in accordions
accordion_geo_ms.update_values(data.loc[data['Estrategia'].isin([strategies.value]) & data['Subestrategia'].isin([sub_strategies.value]), 'Area Geográfica 2'].unique().tolist(), True)
accordion_risk_ms.update_values(data.loc[data['Estrategia'].isin([strategies.value]) & data['Subestrategia'].isin([sub_strategies.value]), 'Perfil de Riesgo'].unique().tolist(), True)

# Row 2
row2 = widgets.HBox([accordion_geo, aux_h, accordion_risk])

#Tab 1

#Output Dropdown
d_stats = widgets.Dropdown(options = ['Patrimonio actual',
                                      'WTD', 'MTD', 'YTD',
                                      'Volatilidad 3M', 'Volatilidad 1Y', 'Downside Vol 3M', 'Downside Vol 1Y',
                                      'Drawdown'], value = 'Patrimonio actual')
d_stats.layout.width = '150px'
d_stats.observe(update_output1, names = 'value')

#Output Bar
xs = OrdinalScale()
ys = LinearScale()
sc = ColorScale()

tooltip_bar = Tooltip(fields=['x', 'y'], formats=['', '0.2f'], labels = ['Fondo', d_stats.value])

bar = Bars(x=[], y=[], scales={'x': xs, 'y': ys, 'color': sc}, color=[], padding=0.5, tooltip=tooltip_bar, interactions = {'hover': 'tooltip'})

xax = Axis(scale=xs, tick_rotate=-20, tick_style = {'font-weight': 'bold'})
yax = Axis(scale=ys, orientation='vertical', tick_format='0.00', grid_lines='solid')
xax_c = ColorAxis(scale=sc, tick_format='0.00')

plot1 = Figure(marks=[bar], axes=[xax, yax, xax_c])
plot1.fig_margin = {'top': 20, 'bottom': 90, 'left': 80, 'right': 30}
plot1.layout.width = '1675px'

tab1 = widgets.VBox([d_stats, plot1])

#Tab 2

# Label X
label_X = widgets.Label(value = 'X')

#X Dropdown
x_stats = widgets.Dropdown(options = ['Patrimonio actual',
                                      'WTD', 'MTD', 'YTD',
                                      'Volatilidad 3M', 'Volatilidad 1Y', 'Downside Vol 3M', 'Downside Vol 1Y',
                                      'Drawdown'], value = 'Patrimonio actual')
x_stats.layout.width = '150px'
x_stats.observe(update_output2, names = 'value')

# Label Y
label_Y = widgets.Label(value = 'Y')

#Y Dropdown
y_stats = widgets.Dropdown(options = ['Patrimonio actual',
                                      'WTD', 'MTD', 'YTD',
                                      'Volatilidad 3M', 'Volatilidad 1Y', 'Downside Vol 3M', 'Downside Vol 1Y',
                                      'Drawdown'], value = 'YTD')
y_stats.layout.width = '150px'
y_stats.observe(update_output2, names = 'value')

#Output Scatter
scatt_xs = LinearScale()
scatt_ys = LinearScale()

tooltip_scatt = Tooltip(fields=['x', 'y'], formats=['0.2f', '0.2f'], labels = [x_stats.value, y_stats.value])

scatt = Scatter(x=[], y=[], names=[],
                scales={'x': scatt_xs, 'y': scatt_ys}, apply_clip=False, tooltip=tooltip_scatt, interactions = {'hover': 'tooltip'})

scatt_xax = Axis(scale=scatt_xs)
scatt_yax = Axis(scale=scatt_ys, orientation='vertical', grid_lines='solid')

p = PolyFit(x_train=[1], y_train=[1], scales={'x': scatt_xs, 'y': scatt_ys})
order_slider = IntSlider(description='Order', value = 1)
link((order_slider, 'value'), (p, 'order'))

# plot2 = Figure(marks=[scatt], axes=[scatt_xax, scatt_yax])
plot2 = Figure(marks=[scatt, p], axes=[scatt_xax, scatt_yax])
plot2.fig_margin = {'top': 20, 'bottom': 20, 'left': 30, 'right': 100}
plot2.layout.width = '1675px'

tab2 = widgets.VBox([widgets.HBox([label_X, x_stats, aux_h, label_Y, y_stats]), plot2])

tab = widgets.Tab(children = [tab1, tab2])
tab.set_title(0, 'Estadísticas')
tab.set_title(1, 'Estadísticas cruzadas')
tab.layout.visibility = 'hidden'

main = widgets.VBox([path, label_overview, row0, aux_v, label_analysis, row1, label_filters, row2, aux_v, tab])
main